In [162]:
import pandas as pd
import numpy as np 
from cleanco import basename
from sklearn.feature_extraction import text
import pycountry
from fuzzywuzzy import fuzz

from difflib import SequenceMatcher
from collections import Counter
pd.set_option('display.max_colwidth', -1)



C:\Users\Xiaoyi WU\AppData\Local\Temp\ipykernel_21940\1310382122.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


### 1. Read Data

In [163]:
test_df=pd.read_csv('./input/test.csv')
test_df.loc[:1,:]

gkgrecordid     date sourcecollectionidentifier sourcecommonname  \
0  20180301144500-205   45:00.0  1                          allafrica.com     
1  20180301144500-2771  45:00.0  1                          angop.ao          

                                                                                                                                          documentidentifier  \
0  http://allafrica.com/stories/201803010741.html                                                                                                              
1  http://www.angop.ao/angola/en_us/noticias/politica/2018/2/9/Angola-gets-information-regarding-Lesotho-situation,07ce6836-496b-4ec3-81c4-1aab3318cca6.html   

  counts  \
0  NaN     
1  NaN     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       locations  \
0  [{location_type=1, location_fullname=Lesotho, location_countrycode=LT, location_adm1code=LT, location_adm2code=, location_latitude=-29.5, location_longitude=28.25, location_featureid=LT, character_offset=625}, {location_type=1, location_fullname=Lesotho, location_countrycode=LT, location_adm1code=LT, location_adm2code=, location_latitude=-29.5, location_longitude=28.25, location_featureid=LT, character_offset=2170}, {location_type=1, location_fullname=Basotho, location_countrycode=LT, location_adm1code=LT, location_adm2code=, location_latitude=-29.5, location_longitude=28.25, location_featureid=LT, character_offset=349}, {location_type=1, location_fullname=Basotho

In [164]:
df=test_df.dropna(subset=['organizations']).reset_index()

In [165]:
# get the organization list in 'organizations' columns
import re
company_ls=[]
for i in range(df.shape[0]):
    str=df.loc[i]['organizations']
    pat = re.compile("organization[=:].*?,")
    for j in [s[13:-1] for s in pat.findall(str)]:
        company_ls.append(j)

In [166]:
indata=pd.DataFrame(data=sorted(company_ls),columns=['name_original'])
indata.head()

,name_original
0,A American Electrical Services
1,A Constitutional Court
2,A Federal High Court
3,A Global Initiative Against Transnational Crime
4,A National University Of Lesotho


#### preprocess nlp data

We need to clean company names to get rid of odd artifacts and other wrinkles. Remove anything in parenthesis, all punctuation, and any extra whitespaces.

In [167]:
import string
import unicodedata
import regex as re

stop = text.ENGLISH_STOP_WORDS

# Helper methods for name cleaning.
def preprocess_nlp(row):
        row = row.lower()
        row = row.strip()
        row = re.sub(r'\(.*\)', '', row)
        row = row.translate(str.maketrans('', '', string.punctuation))
        row =' '.join(row.split())
        row = unicodedata.normalize('NFKD', row).encode('ASCII', 'ignore').decode()
        row =  ' '.join(word.lower() for word in row.split() if word not in stop)
        return row

In [168]:
indata['name_clean']=indata['name_original'].apply(preprocess_nlp)
print(indata.shape)
indata.head()

(14847, 2)


,name_original,name_clean
0,A American Electrical Services,american electrical services
1,A Constitutional Court,constitutional court
2,A Federal High Court,federal high court
3,A Global Initiative Against Transnational Crime,global initiative transnational crime
4,A National University Of Lesotho,national university lesotho


In [169]:
data_unique = pd.DataFrame( data = pd.unique(indata['name_clean']) , columns=['name_clean'])
print(data_unique.shape)
data_unique.head()

(4508, 1)


,name_clean
0,american electrical services
1,constitutional court
2,federal high court
3,global initiative transnational crime
4,national university lesotho


### remove noise words(or stopwords) in organization names

In [170]:
# prepare noise words(or stopwords) in organization names such as organizations, groups, they are so general that will influence clustering result
def word_count(x):
    word_list = x.split()
    number_of_words = len(word_list)
    return(number_of_words)

In [171]:
data_unique['word_count']= data_unique['name_clean'].apply(lambda x: word_count(x))

In [172]:
data_unique.loc[data_unique['word_count']==1].to_csv('./output/one_word.csv')
data_unique.shape

(4508, 2)

In [173]:
# todo: hand sort data for noise words
noise_word = pd.read_csv('./input/stop_word_in_company_name.csv')
noise_word


,name_clean,lenth
0,centre,1
1,commission,1
2,company,1
3,council,1
4,court,1
5,beers,1
6,solutions,1
7,express,1
8,department,1
9,group,1


In [174]:
outdata_no_stopwords=data_unique[~data_unique['name_clean'].isin(noise_word['name_clean'])].reset_index()
outdata_no_stopwords.shape

(4481, 3)

### remove country name

In [175]:
file = open("./raw data/country_spellings.txt")
lines = file.readlines()

country_ls=[]
for l in lines:
    line=l.strip()
    line_2=line.replace('"',"")
    pat = re.compile("sname(?:==|:)(.*?);")
    # print(line_2)
    for i in pat.findall(line_2):
        country_ls.append(i.lower())
    # break
len(country_ls)

640

In [176]:
for i in pycountry.countries:
    country_ls.append(i.alpha_2.lower())
    country_ls.append(i.alpha_3.lower())
    country_ls.append(i.name.lower())
len(country_ls)

1387

In [177]:
country_ls.sort(reverse=True)
country_df=pd.DataFrame(data=country_ls,columns=['country_name'])
country_df.drop_duplicates(subset ='country_name',inplace=True)
print(country_df.shape)
country_df.head()


(911, 1)


,country_name
0,åland islands
1,zwe
2,zw
3,zmb
4,zm


In [178]:
country_df['country_name_clean']=country_df['country_name'].apply(preprocess_nlp)
country_df.head()

,country_name,country_name_clean
0,åland islands,aland islands
1,zwe,zwe
2,zw,zw
3,zmb,zmb
4,zm,zm


In [179]:
outdata_no_country=outdata_no_stopwords[~outdata_no_stopwords['name_clean'].isin(country_df['country_name_clean'])].reset_index()
print(outdata_no_country.shape)
outdata_no_country.to_csv('.\output\outdata_no_country.csv')

(4479, 4)


In [180]:
outdata_no_stopwords[outdata_no_stopwords['name_clean'].isin(country_df['country_name_clean'])].reset_index()

,level_0,index,name_clean,word_count
0,3556,3576,russian federation,2
1,4143,4167,united states,2


### calculate the similarity

In [181]:
def fuzz_similarity(data): 
    company_names=data.name_clean.to_list()
    similarity_array = np.ones((len(company_names),(len(company_names))))*100

    for i in range(1,len(company_names)):
        for j in range(i):
            s_1 = fuzz.token_set_ratio(company_names[i],company_names[j])+ 1e-8
            s_2 = fuzz.partial_ratio(company_names[i],company_names[j])+ 1e-8
            similarity_array[i][j] = 2*s_1*s_2/(s_1+s_2)
    for i in range(len(company_names)):
        for j in range(i+1,len(company_names)):
            similarity_array[i][j] = similarity_array[j][i]
    np.fill_diagonal(similarity_array, 100)
    return similarity_array
    


In [182]:
#similarity matrix
similarity=fuzz_similarity(outdata_no_country)
df=pd.DataFrame(data=similarity,columns=outdata_no_country.name_clean.to_list(),index=outdata_no_country.name_clean.to_list())
df

,american electrical services,constitutional court,federal high court,global initiative transnational crime,national university lesotho,university pittsburgh,aaron,abba palestinian authority,abrahams kiewitz,academy award,...,zimbabwe national army,zimbabwe national chamber,zimbabwe national statistics agency,zimbabwe nurses association,zimbabwe revenue authority zimra,zimbabwe union journalists,zimbabwean african people union zapu,zimbabweans department home affairs,zion christian church bishop,zipline international
american electrical services,100.000000,36.164384,39.000000,37.975000,38.337662,30.870968,24.827586,39.341772,34.742857,29.966667,...,40.493827,42.352941,41.976190,33.313433,34.434783,32.307692,46.494624,36.891892,38.769231,39.443038
constitutional court,36.164384,100.000000,43.494253,47.628866,47.812500,36.756757,22.857143,36.000000,20.390244,23.489362,...,41.619048,44.444444,40.000000,36.438356,30.491803,41.785714,31.718750,22.500000,43.181818,45.450549
federal high court,39.000000,43.494253,100.000000,28.448276,31.968750,31.968750,12.413793,35.154930,27.789474,24.408163,...,28.965517,33.000000,25.254902,29.423729,37.440000,37.052632,29.466667,31.428571,33.913043,34.434783
global initiative transnational crime,37.975000,47.628866,28.448276,100.000000,42.447059,35.888889,34.285714,41.493976,26.407407,29.419355,...,46.891089,50.076923,40.902439,36.373333,32.895522,38.181818,34.434783,33.970588,37.973684,53.839286
national university lesotho,38.337662,47.812500,31.968750,42.447059,100.000000,65.984848,28.860760,39.900000,35.323944,23.958333,...,46.234043,41.860465,50.376238,43.494253,34.492754,42.000000,35.972222,32.515152,40.493827,43.181818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zimbabwe union journalists,32.307692,41.785714,37.052632,38.181818,42.000000,33.492537,25.762712,40.780488,26.851852,21.954545,...,67.496296,61.463415,61.824000,57.495652,56.460177,100.000000,55.928571,49.494950,37.493333,44.911111
zimbabwean african people union zapu,46.494624,31.718750,29.466667,34.434783,35.972222,24.360000,26.666667,42.211765,28.862069,35.323944,...,59.000000,57.460870,49.980000,52.981132,51.456311,55.928571,100.000000,54.981818,38.493507,44.800000
zimbabweans department home affairs,36.891892,22.500000,31.428571,33.970588,32.515152,30.870968,30.769231,40.444444,36.373333,41.619048,...,57.982906,52.830189,46.000000,52.000000,51.980769,49.494950,54.981818,100.000000,36.972973,36.972973
zion christian church bishop,38.769231,43.181818,33.913043,37.973684,40.493827,35.323944,30.000000,39.443038,28.862069,21.395349,...,31.500000,36.756757,36.891892,38.775000,32.492308,37.493333,38.493507,36.972973,100.000000,37.493333


In [183]:
df.to_csv('df.csv')

### clustering based on similarity

In [184]:
import scipy
n_components, labels=scipy.sparse.csgraph.connected_components((similarity > 90).astype(int),directed=False)
labels

array([   0,    1,    2, ...,   39, 2809,    4])

In [185]:
similarity[990]

array([26.26415095, 41.7857143 , 35.75000001, ..., 33.00000001,
       30.48387098, 36.02702704])

In [186]:
outdata_no_country['cluster']=labels.tolist() 
outdata_no_country

,level_0,index,name_clean,word_count,cluster
0,0,0,american electrical services,3,0
1,1,1,constitutional court,2,1
2,2,2,federal high court,3,2
3,3,3,global initiative transnational crime,4,3
4,4,4,national university lesotho,3,4
...,...,...,...,...,...
4474,4476,4503,zimbabwe union journalists,3,2807
4475,4477,4504,zimbabwean african people union zapu,5,2808
4476,4478,4505,zimbabweans department home affairs,4,39
4477,4479,4506,zion christian church bishop,4,2809


In [187]:
outdata_no_country.to_csv('tmp_4.csv')

3. Assigning Standard Names

In [188]:
def standard_name(df_eval):
    d_standard_name = {}
    for cluster  in df_eval.cluster.unique():
        names= df_eval[df_eval['cluster'] == cluster].name_clean.to_list()
        l_common_substring = []
        if len(names)>1:
            for i in range(len(names)):
                for j in range(i+1,len(names)):
                    seqMatch = SequenceMatcher(None, names[i], names[j])
                    match = seqMatch.find_longest_match(0, len(names[i]), 0, len(names[j]))
                    if (match.size!=0):
                        l_common_substring.append(names[i][match.a:match.a+match.size].strip())
            n = len(l_common_substring)
            counts = Counter(l_common_substring)
            get_mode = dict(counts)
            mode = [k for k,v in get_mode.items() if v == max(list(counts.values()))]
            d_standard_name[cluster] = ";".join(mode)

        else:
            d_standard_name[cluster] = names[0]
            
    df_standard_names = pd.DataFrame(list(d_standard_name.items()),columns=['cluster','standard_name'])
    df_eval = df_eval.merge(df_standard_names,on='cluster',how='left')
    return df_standard_names,df_eval

In [189]:
df_standard_names,df_eval = standard_name(df)
df_eval



AttributeError: 'DataFrame' object has no attribute 'cluster'

### frequency count

In [ ]:
outdata_gdelt.to_csv('./output/gdelt_list.csv')